# Créer votre premier mécanisme de recommandation pour le commerce électronique

Ce bloc-notes vous guide dans les étapes de la création d'un groupe de jeux de données de domaine et d'un mécanisme de recommandation qui renvoie des recommandations de produits basées sur les données générées pour notre jeu de données de magasin de détail fictif. L'objectif est de recommander des produits pertinents en fonction d'un utilisateur donné.

Ces données synthétiques proviennent du [projet Retail Demo Store](https://github.com/aws-samples/retail-demo-store). Suivez le lien pour en savoir plus sur les données et les utilisations potentielles.

# Comment utiliser le bloc-notes

Le code est divisé en cellules comme celui donné ci-dessous. Il y a un bouton Exécuter triangulaire en haut de cette page qui vous permet d'exécuter chaque cellule et de passer à la suivante, ou vous pouvez appuyer sur `Shift` + `Enter` lorsque vous êtes dans la cellule pour l'exécuter et passer à la suivante.

Lorsqu'une cellule est en cours d'exécution, vous remarquerez une ligne sur le coté affichant une `*` pendant que la cellule est opérationnelle ou elle se mettra à jour sous forme de nombre pour indiquer la dernière cellule qui a terminé l'exécution après avoir achevé l'exécution de tous les codes dans une cellule.

Il suffit de suivre les instructions ci-dessous et d'exécuter les cellules pour commencer avec Amazon Personalize en utilisant des mécanismes de recommandation optimisés pour les cas.

## Importations
Python est livré avec une large gamme de bibliothèques. Nous devons les importer, ainsi que celles qui sont installées pour nous aider, comme [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK pour python) et [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), qui sont des outils essentiels pour la science des données.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

Ensuite, vous devez vérifier que votre environnement peut communiquer avec Amazon Personalize. Les lignes ci-dessous servent à cela.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Spécifier un compartiment S3 et un emplacement de sortie des données

Amazon Personalize nécessitera d'utiliser un compartiment S3 comme source de vos données. Le code ci-dessous créera un compartiment avec un `bucket_name` unique.

Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize. 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
    
except Exception as e:
    print (e)
    print("Bucket already exists. Using bucket", bucket_name)

## Télécharger, préparer et charger les données d'entraînement

Nous devons d'abord télécharger les données (données d'entraînement). Dans ce tutoriel, nous allons utiliser l'historique à partir du jeu de données d'un magasin de détail. Le jeu de données contient l'identificateur de l'utilisateur, l'identificateur des articles, l'interaction entre les clients et les articles et l'heure à laquelle cette interaction a eu lieu (Timestamp). 

### Télécharger et explorer le jeu de données

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

Le jeu de données téléchargé s'appelle Electronics_Store_purchase_history.csv

Apprenez-en davantage sur le jeu de données en vous reportant à ses caractéristiques.

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.EVENT_TYPE.value_counts()

In [ ]:
def convert_event_type(event_type_in_some_format):
    if(event_type_in_some_format == "ProductViewed"):
        return "View"
    if(event_type_in_some_format == "OrderCompleted"):
        return "Purchase"
    else:
        return event_type_in_some_format

df['EVENT_TYPE'] = df['EVENT_TYPE'].apply(convert_event_type)

In [ ]:
df.EVENT_TYPE.value_counts()

Les mécanismes de recommandation ECOMMERCE exigent que vous fournissiez des valeurs EVENT_TYPE spécifiques, afin de comprendre le contexte d'une interaction. Par conséquent, nous allons modifier la colonne EVENTYPE de nos interactions.

In [ ]:
df.info()

A partir des 2 cellules ci-dessus, nous avons appris que nos données ont 9 colonnes, 675 004 lignes et que les en-têtes sont : ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP et DISCOUNT.

Pour être compatible avec un schéma d'interactions Amazon Personalize, ce jeu de données nécessite des titres de colonne compatibles avec les noms de colonne par défaut d'Amazon Personalize (des informations sur les noms de colonne sont disponibles [ici](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)).

## Préparer les données d'interactions


### Supprimer des colonnes

Certaines colonnes dans ce jeu de données n'apporteraient pas de valeur ajoutée à notre modèle et doivent donc être supprimées. Colonnes telles que *discount*.

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

Dans les cellules ci-dessous, nous allons écrire nos données nettoyées dans le fichier nommé " final_training_data.csv

In [ ]:
df.to_csv("cleaned_training_data.csv")

### Charger vers S3
Maintenant que nos données d'entraînement sont prêtes pour Amazon Personalize, l'étape suivante consiste à les charger vers le compartiment S3 créé précédemment.

In [ ]:
interactions_file_path = 'cleaned_training_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path


## Configurer un compartiment S3 et un rôle IAM

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.


## Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

Remarque : Assurez-vous que le rôle que vous utilisez pour exécuter le code dans ce bloc-notes dispose des autorisations nécessaires pour modifier la politique du compartiment S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## Créer et attendre un groupe de jeux de données
Le groupe le plus important de Personalize est le groupe de jeu de données qui permet d'isoler vos données, vos outils de suivi des événements, vos solutions, vos systèmes de recommandation et vos campagnes. Regroupement de choses qui partagent un ensemble commun de données. N'hésitez pas à modifier le nom ci-dessous si vous le souhaitez.

### Créer un groupe de jeux de données

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Attendre que le groupe de jeu de données ait le statut ACTIF
Avant de pouvoir utiliser le groupe de jeu de données dans les articles ci-dessous, il doit être actif; exécutez la cellule ci-dessous et attendez qu'il soit actif.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Créer un schéma d'interactions
L'un des principaux articles permettant à Personalize de comprendre vos données provient du schéma défini ci-dessous. Cette configuration indique au service comment traiter les données fournies par votre fichier CSV. Notez que les colonnes et les types s'alignent sur les articles présents dans le fichier que vous avez créé ci-dessus.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Créer des jeux de données
Après avoir créé le groupe, vous devez créer les jeux de données proprement dits.

### Créer un jeu de données d'interactions

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Créer un rôle Personalize
De plus, Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin d'avoir les autorisations d'exécuter certaines tâches. Les lignes ci-dessous les accordent.

Remarque : Assurez-vous que le rôle que vous utilisez pour exécuter le code dans ce bloc-notes dispose des autorisations nécessaires pour créer un rôle.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importer les données
Vous avez précédemment créé le groupe de jeu de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données de S3 dans Amazon Personalize pour les utiliser dans la création de votre modèle.
### Créer une tâche d'importation de jeux de données d'interactions

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

Attendre que la tâche d'importation de données ait le statut ACTIF
La tâche d'importation peut durer un certain temps ; veuillez patienter jusqu'à ce que vous voyiez qu'elle est active ci-dessous.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Choisir des cas d'utilisation d'un mécanisme de recommandation

Chaque domaine a des cas d'utilisation différents. Lorsque vous créez un mécanisme de recommandation, vous le créez pour un cas d'utilisation spécifique, et chaque cas d'utilisation a des exigences différentes pour obtenir des recommandations.


In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

Nous allons créer un mécanisme de recommandation du type "Les clients qui ont vu X ont aussi vu". Ce mécanisme de recommandation donne des recommandations pour les articles que les clients ont également consultés en fonction d'un article que vous spécifiez. Dans ce cas d'utilisation, Amazon Personalize filtre automatiquement les articles achetés par l'utilisateur en fonction du userId que vous spécifiez et des événements `Purchase`.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'viewed_x_also_viewed_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  datasetGroupArn = dataset_group_arn
)
viewed_x_also_viewed_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Nous allons créer un deuxième mécanisme de recommandation du type "Recommandé pour vous". Ce type de recommandation offre des recommandations personnalisées pour des articles en fonction d'un utilisateur que vous spécifiez. Dans ce cas d'utilisation, Amazon Personalize filtre automatiquement les articles achetés par l'utilisateur en fonction du userId que vous spécifiez et des événements `Purchase`.

[Plus de cas d'utilisation par domaine](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Nous attendons que les mécanismes de recommandations aient fini de créer et qu'ils aient le statut `ACTIVE`. Nous vérifions régulièrement l'état du mécanisme de recommandation

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours

while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = viewed_x_also_viewed_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(viewed_x_also_viewed_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(viewed_x_also_viewed_arn))
        

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Customers who viewed X also viewed" Recommender build is still in progress')
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Recommended for you" Recommender build is still in progress')
        
    time.sleep(60)

## Obtenir des recommandations avec un mécanisme de recommandation
Maintenant que les mécanismes de recommandation ont été entraînés, examinons les recommandations que nous pouvons obtenir pour nos utilisateurs.

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

Obtenons des recommandations en utilisant le mécanisme de recommandation "Les clients qui ont vu X ont aussi vu" :

In [ ]:
# use a random valid id for a quick sanity check, modify the line of code bellow to a valid id in your dataset
get_item_by_id("c72257d4-430b-4eb7-9de3-28396e593381", items_df)

In [ ]:
# First pick a user
test_user_id = "777"

# Select a random item
test_item_id = "8fbe091c-f73c-4727-8fe7-d27eabd17bea" # a random item: 8fbe091c-f73c-4727-8fe7-d27eabd17bea

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = test_item_id,
    userId = test_user_id,
    numResults = 10
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_item_by_id(test_item_id, items_df)])

pd.options.display.max_rows =10
display(user_recommendations_df)

Obtenez des recommandations du mécanisme de recommandation qui retourne "Recommandé pour vous" :

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)


user_recommendations_df = pd.DataFrame(recommendation_list, columns = [test_user_id])

pd.options.display.max_rows =20
display(user_recommendations_df)

## Examiner
En utilisant les codes ci-dessus, vous avez entraîné un modèle de deep learning pour générer des recommandations d'articles en fonction du comportement antérieur de l'utilisateur. Vous avez créé deux mécanismes de recommandation pour deux cas d'utilisation fondamentaux. 
À l'avenir, vous pourrez adapter ce code pour créer d'autres mécanismes de recommandations.

## Remarques pour le bloc-notes suivant :
Vous aurez besoin de quelques valeurs pour le bloc-notes suivant. Exécutez la cellule ci-dessous pour les stocker, afin qu'elles puissent être utilisées dans le bloc-notes `Clean_Up_Resources.ipynb`.

Toutes les données stockées pour ces variables seront remplacées et définies aux valeurs spécifiées dans ce bloc-notes. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

Si vous avez exécuté le bloc-notes `Building_Your_First_Recommender_Video_On_Demand.ipynb`, veillez à réexécuter l'étape précédente du bloc-notes `Building_Your_First_Recommender_Video_On_Demand.ipynb` et `Clean_Up_Resources.ipynb` pour supprimer les ressources créées dans ce bloc-notes après avoir exécuté `Clean_Up_Resources.ipynb` avec les ressources créées ici.